In [1]:
import numpy as np
import skfda
from skfda.representation.basis import FDataBasis
from skfda.datasets._real_datasets import fetch_growth
from matplotlib import pyplot

In [2]:
dataset = fetch_growth()
fd = dataset['data']
y = dataset['target']

In [15]:
print(fd)

Data set:    [[[ 81.3]
  [ 84.2]
  [ 86.4]
  ...
  [193.8]
  [194.3]
  [195.1]]

 [[ 76.2]
  [ 80.4]
  [ 83.2]
  ...
  [176.1]
  [177.4]
  [178.7]]

 [[ 76.8]
  [ 79.8]
  [ 82.6]
  ...
  [170.9]
  [171.2]
  [171.5]]

 ...

 [[ 68.6]
  [ 73.6]
  [ 78.6]
  ...
  [166. ]
  [166.3]
  [166.8]]

 [[ 79.9]
  [ 82.6]
  [ 84.8]
  ...
  [168.3]
  [168.4]
  [168.6]]

 [[ 76.1]
  [ 78.4]
  [ 82.3]
  ...
  [168.6]
  [168.9]
  [169.2]]]
sample_points:    [array([ 1.  ,  1.25,  1.5 ,  1.75,  2.  ,  3.  ,  4.  ,  5.  ,  6.  ,
        7.  ,  8.  ,  8.5 ,  9.  ,  9.5 , 10.  , 10.5 , 11.  , 11.5 ,
       12.  , 12.5 , 13.  , 13.5 , 14.  , 14.5 , 15.  , 15.5 , 16.  ,
       16.5 , 17.  , 17.5 , 18.  ])]
time range:    [[ 1. 18.]]


from here onwards is the implementation that should be inside the fit function

In [5]:
fd_data = np.squeeze(fd.data_matrix)

In [8]:
n_samples, n_points_discretization = fd_data.shape

In [14]:
k_estimated = fd_data @ np.transpose(fd_data)/n_samples

In [16]:
fd.sample_points

[array([ 1.  ,  1.25,  1.5 ,  1.75,  2.  ,  3.  ,  4.  ,  5.  ,  6.  ,
         7.  ,  8.  ,  8.5 ,  9.  ,  9.5 , 10.  , 10.5 , 11.  , 11.5 ,
        12.  , 12.5 , 13.  , 13.5 , 14.  , 14.5 , 15.  , 15.5 , 16.  ,
        16.5 , 17.  , 17.5 , 18.  ])]

In [10]:
print(n_points_discretization)

31


In [17]:
fd.sample_points[0]

array([ 1.  ,  1.25,  1.5 ,  1.75,  2.  ,  3.  ,  4.  ,  5.  ,  6.  ,
        7.  ,  8.  ,  8.5 ,  9.  ,  9.5 , 10.  , 10.5 , 11.  , 11.5 ,
       12.  , 12.5 , 13.  , 13.5 , 14.  , 14.5 , 15.  , 15.5 , 16.  ,
       16.5 , 17.  , 17.5 , 18.  ])

what weight vectors should we use?

In [21]:
weights = np.diff(fd.sample_points[0])
weights = np.append(weights, [weights[-1]])

In [25]:
weights_matrix = np.diag(weights)

In [29]:
final_matrix = fd_data @ np.sqrt(weights_matrix) / np.sqrt(n_samples)

In [30]:
u, s, vh = np.linalg.svd(final_matrix, full_matrices=True, compute_uv=True)

observe that we obtain the same by decomposing using eig directly

In [33]:
print(vh)

[[-6.46348074e-02 -6.80259397e-02 -7.09800076e-02 -7.36136232e-02
  -1.52001225e-01 -1.66509506e-01 -1.79517115e-01 -1.91597131e-01
  -2.03391330e-01 -2.14297296e-01 -1.58737520e-01 -1.62341098e-01
  -1.65953620e-01 -1.69411393e-01 -1.72901084e-01 -1.76607524e-01
  -1.80405503e-01 -1.84322127e-01 -1.88237453e-01 -1.92028262e-01
  -1.95624282e-01 -1.98937513e-01 -2.01862032e-01 -2.04288111e-01
  -2.06225610e-01 -2.07614907e-01 -2.08673474e-01 -2.09402232e-01
  -2.09908501e-01 -2.10248402e-01 -2.10603645e-01]
 [-4.44566582e-03 -1.39027900e-02 -1.98234062e-02 -2.36439972e-02
  -7.00284155e-02 -6.38249167e-02 -8.46637858e-02 -1.23326597e-01
  -1.67692729e-01 -1.48972480e-01 -1.00280297e-01 -1.03060109e-01
  -1.06129666e-01 -1.17194973e-01 -1.30543371e-01 -1.59769501e-01
  -1.95693665e-01 -2.26458587e-01 -2.35368517e-01 -2.07751450e-01
  -1.45802525e-01 -5.94257836e-02  3.11530544e-02  1.18896274e-01
   1.89969739e-01  2.42224219e-01  2.80701979e-01  3.06450634e-01
   3.22102688e-01  3.3391

In [31]:
print(s**2)

[3.34718386e+05 1.02805310e+02 2.71985229e+01 9.39226467e+00
 3.67840534e+00 1.65819915e+00 1.38068476e+00 1.19223015e+00
 6.59966620e-01 5.06723349e-01 3.01234518e-01 2.57601625e-01
 1.97639361e-01 1.47572675e-01 1.01509765e-01 8.28738857e-02
 5.81587402e-02 3.86702709e-02 2.66249248e-02 2.18573322e-02
 1.58645660e-02 1.10728476e-02 9.07623198e-03 6.87504706e-03
 4.38147552e-03 3.70917729e-03 3.18338768e-03 2.42622590e-03
 1.96628521e-03 1.53257970e-03 9.04160622e-04]


In [32]:
np.linalg.eig(np.transpose(final_matrix) @ final_matrix)

(array([3.34718386e+05, 1.02805310e+02, 2.71985229e+01, 9.39226467e+00,
        3.67840534e+00, 1.65819915e+00, 1.38068476e+00, 1.19223015e+00,
        6.59966620e-01, 5.06723349e-01, 3.01234518e-01, 2.57601625e-01,
        1.97639361e-01, 1.47572675e-01, 1.01509765e-01, 8.28738857e-02,
        5.81587402e-02, 3.86702709e-02, 2.66249248e-02, 2.18573322e-02,
        1.58645660e-02, 1.10728476e-02, 9.07623198e-03, 6.87504706e-03,
        9.04160626e-04, 4.38147552e-03, 1.53257970e-03, 1.96628521e-03,
        2.42622591e-03, 3.70917729e-03, 3.18338768e-03]),
 array([[-6.46348074e-02, -4.44566582e-03, -1.26672276e-01,
          2.07149930e-01, -3.24804309e-01,  1.27452666e-01,
          5.27725144e-01,  2.20895955e-01,  1.80313174e-01,
         -2.92834877e-02,  4.29046786e-01, -2.58491690e-01,
         -2.00456056e-01, -1.50566848e-01,  1.88612148e-01,
          2.40490432e-01,  1.51750779e-01, -2.48569466e-02,
         -4.63206396e-02,  3.58172251e-02, -2.49448747e-02,
          8.471825